In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets

# Load Your Own Cifar 10 model

In [ ]:
loaded_model = tf.keras.models.load_model('cifar10.h5')
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [ ]:
loaded_model.layers

# Lets only take layers till the last CNN block from this model

In [ ]:
inputs = keras.Input((32,32,3))
x = inputs
for layer in loaded_model.layers[0:-3]:
  x = loaded_model.get_layer(layer.name)(x)

custom_model = keras.Model(inputs,x)
custom_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_______________________________________________________

In [ ]:
# lets check if there is weights
custom_model.layers[1].get_weights()

# there is indeed weights

# Lets load our cats and dogs

In [ ]:
import pathlib
data_dir = pathlib.Path('../data/dogs_vs_cats/')

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1.0/255.)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1.0/255.)

batch_size = 32
img_height = 32
img_width = 32
train_ds = train_gen.flow_from_directory(
  str(data_dir) + '/train',
  seed=123,
  target_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = train_gen.flow_from_directory(
  str(data_dir) + '/test',
  seed=123,
  target_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_indices
print(class_names)

Found 40 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}


In [ ]:
### lets add our very own customizations
## shape of 180,180
## flatten, dense of 128, last layer softmax

inputs = keras.Input((32,32,3))
x = inputs

# freeze all layers of the loaded model
# BAM !!!
loaded_model.trainable = False

for layer in loaded_model.layers[0:-3]:
  x = loaded_model.get_layer(layer.name)(x)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(len(class_names))(x)
custom_model = keras.Model(inputs,x)

custom_model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

custom_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_7 (Flatten)          (None, 1024)              0  

In [ ]:
epochs=10
history = custom_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
2/2 [==============================] - 1s 511ms/step - loss: 0.8767 - accuracy: 0.5000 - val_loss: 0.7703 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 0s 167ms/step - loss: 0.7512 - accuracy: 0.5000 - val_loss: 0.6340 - val_accuracy: 0.6500
Epoch 3/10
2/2 [==============================] - 0s 293ms/step - loss: 0.7083 - accuracy: 0.5250 - val_loss: 0.7010 - val_accuracy: 0.6000
Epoch 4/10
2/2 [==============================] - 0s 303ms/step - loss: 0.6769 - accuracy: 0.5500 - val_loss: 0.7388 - val_accuracy: 0.5500
Epoch 5/10
2/2 [==============================] - 0s 186ms/step - loss: 0.6573 - accuracy: 0.5250 - val_loss: 0.6909 - val_accuracy: 0.5500
Epoch 6/10
2/2 [==============================] - 1s 435ms/step - loss: 0.6435 - accuracy: 0.7250 - val_loss: 0.7449 - val_accuracy: 0.4000
Epoch 7/10
2/2 [==============================] - 1s 630ms/step - loss: 0.5721 - accuracy: 0.7750 - val_loss: 0.7482 - val_accuracy: 0.4500
Epoch 8/10
2/2 [====

# What just happened ?? We reused an existing model, and added our customization !
# the existing model preserves its weights and biases